In [2]:
from __future__ import print_function

In [44]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from nltk.probability import FreqDist
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from sklearn.metrics import confusion_matrix
import seaborn as sns
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics
from sklearn.model_selection import train_test_split
from matplotlib import cm
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.naive_bayes import MultinomialNB
from nltk.tokenize import RegexpTokenizer
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import re
import pyLDAvis
import pyLDAvis.sklearn
pyLDAvis.enable_notebook()
from sklearn.datasets import fetch_20newsgroups
from sklearn.decomposition import LatentDirichletAllocation

In [45]:
pd.set_option('display.max_columns', None)

In [46]:
def drop_col(df):
    new_df = df.drop(['id', 'conversation_id', 'created_at','timezone','user_id','place','urls','photos','replies_count','retweets_count', 'likes_count','cashtags',
                      'retweet','quote_url','video','near','geo','source','user_rt_id','user_rt','retweet_id','retweet_date','translate','trans_src','trans_dest'], axis=1)
    return new_df

In [47]:
buttigieg = pd.read_csv('buttigieg.csv')
buttigieg = drop_col(buttigieg)

warren = pd.read_csv('warren.csv')
warren = drop_col(warren)

bernie = pd.read_csv('bernie.csv')
bernie = drop_col(bernie)

biden = pd.read_csv('biden.csv')
biden = drop_col(biden)

amy = pd.read_csv('klobuchar.csv')
amy = drop_col(amy)

bloomberg = pd.read_csv('bloomberg.csv')
bloomberg = drop_col(bloomberg)

trump = pd.read_csv('trump.csv')
trump = drop_col(trump)

/Users/Robert/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [48]:
candidates = [bernie, buttigieg, amy, warren, biden, bloomberg, trump]

In [49]:
for democrat in candidates:
    democrat.tweet = democrat.tweet.str.replace(r'https?:\/\/.*\s?', '')
    democrat.tweet = democrat.tweet.str.replace(r'pic\.twitter\.com\/.*\s?', '')

In [50]:
all_tweets = list(buttigieg['tweet']) + list(warren['tweet']) + list(bernie['tweet']) + list(biden['tweet']) + list(amy['tweet']) + list(bloomberg['tweet']) + list(trump['tweet'])

In [51]:
tf_vectorizer = CountVectorizer(strip_accents = 'unicode',
                                stop_words = 'english',
                                lowercase = True,
                                token_pattern = r'\b[a-zA-Z]{3,}\b',
                                max_df = 0.5, 
                                min_df = 10)
dtm_tf = tf_vectorizer.fit_transform(all_tweets)
print(dtm_tf.shape)

(101396, 10714)


In [52]:
tfidf_vectorizer = TfidfVectorizer(**tf_vectorizer.get_params())
dtm_tfidf = tfidf_vectorizer.fit_transform(all_tweets)
print(dtm_tfidf.shape)

/Users/Robert/opt/anaconda3/lib/python3.7/site-packages/sklearn/feature_extraction/text.py:1817: UserWarning: Only (<class 'numpy.float64'>, <class 'numpy.float32'>, <class 'numpy.float16'>) 'dtype' should be used. <class 'numpy.int64'> 'dtype' will be converted to np.float64.
  UserWarning)


(101396, 10714)


In [53]:
lda_tf = LatentDirichletAllocation(n_components=7, random_state=0)
lda_tf.fit(dtm_tf)

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
                          evaluate_every=-1, learning_decay=0.7,
                          learning_method='batch', learning_offset=10.0,
                          max_doc_update_iter=100, max_iter=10,
                          mean_change_tol=0.001, n_components=7, n_jobs=None,
                          perp_tol=0.1, random_state=0, topic_word_prior=None,
                          total_samples=1000000.0, verbose=0)

In [54]:
lda_tfidf = LatentDirichletAllocation(n_components=7, random_state=0)
lda_tfidf.fit(dtm_tfidf)

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
                          evaluate_every=-1, learning_decay=0.7,
                          learning_method='batch', learning_offset=10.0,
                          max_doc_update_iter=100, max_iter=10,
                          mean_change_tol=0.001, n_components=7, n_jobs=None,
                          perp_tol=0.1, random_state=0, topic_word_prior=None,
                          total_samples=1000000.0, verbose=0)

In [55]:
pyLDAvis.sklearn.prepare(lda_tf, dtm_tf, tf_vectorizer)

/Users/Robert/opt/anaconda3/lib/python3.7/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0      0.108706 -0.122830       1        1  21.679676
3      0.091454  0.051804       2        1  18.574347
5      0.029349 -0.119292       3        1  16.847339
4      0.044378  0.124988       4        1  14.668264
1      0.062653 -0.127197       5        1  11.406967
2      0.094711  0.185500       6        1  11.200880
6     -0.431251  0.007027       7        1   5.622528, topic_info=     Category          Freq       Term         Total  loglift  logprob
948   Default  12425.000000      biden  12425.000000  30.0000  30.0000
5141  Default  12176.000000        joe  12176.000000  29.0000  29.0000
1036  Default  19295.000000  bloomberg  19295.000000  28.0000  28.0000
7029  Default   9442.000000       pete   9442.000000  27.0000  27.0000
915   Default  12294.000000     bernie  12294.000000  26.0000  26.0000
...       ...           ...        ...           ...      ...      ...
7029   Topic7    598.431542       pete   9442.850981   0.1197  -4.6736
5018   Topic7    550.253763       iowa   8026.019726   0.1983  -4.7575
8409   Topic7    422.673722    sanders   7865.553888  -0.0453  -5.0213
2597   Topic7    283.858085        des    801.500436   1.8404  -5.4194
6037   Topic7    280.925410    michael   2654.371306   0.6325  -5.4298

[510 rows x 6 columns], token_table=       Topic      Freq         Term
term                               
11         5  0.988271          abd
36         3  0.993212        abuse
53         3  0.997256       access
111        4  0.979134      activos
118        4  0.980477  actualmente
...      ...       ...          ...
10674      5  0.100468        years
10674      6  0.098715        years
10687      4  0.821014         york
10687      6  0.134414         york
10687      7  0.044805         york

[1147 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 4, 6, 5, 2, 3, 7])

In [41]:
pyLDAvis.sklearn.prepare(lda_tfidf, dtm_tfidf, tfidf_vectorizer)

/Users/Robert/opt/anaconda3/lib/python3.7/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
5      0.051871 -0.046871       1        1  7.672276
8      0.010234  0.140011       2        1  7.328722
15     0.047427  0.101747       3        1  6.938780
17     0.087742 -0.033101       4        1  6.431456
0      0.089075 -0.030586       5        1  6.383581
3      0.090139 -0.007569       6        1  6.044034
10     0.077394 -0.018901       7        1  5.579251
11     0.065295 -0.059887       8        1  4.949785
7      0.035539  0.029724       9        1  4.880482
16     0.039554  0.062299      10        1  4.782182
12     0.006863 -0.231115      11        1  4.613487
4      0.033315  0.060957      12        1  4.505008
1      0.052954  0.006192      13        1  4.316478
18     0.045571 -0.039380      14        1  4.316023
13     0.052905  0.003290      15        1  4.227005
2      0.034777  0.046011      16        1  4.198903
14    -0.357330 -0.027457      17        1  3.449891
9     -0.063037  0.101116      18        1  3.256326
19    -0.158432 -0.161980      19        1  3.158089
6     -0.241855  0.105500      20        1  2.968241, topic_info=     Category         Freq       Term        Total  loglift  logprob
1036  Default  3023.000000  bloomberg  3023.000000  30.0000  30.0000
7029  Default  1642.000000       pete  1642.000000  29.0000  29.0000
9184  Default   377.000000      stone   377.000000  28.0000  28.0000
7674  Default   397.000000        que   397.000000  27.0000  27.0000
8280  Default   284.000000      roger   284.000000  26.0000  26.0000
...       ...          ...        ...          ...      ...      ...
7029  Topic20    51.764692       pete  1642.078465   0.0602  -5.1630
5018  Topic20    48.012992       iowa  1340.070139   0.1882  -5.2382
6412  Topic20    38.623980        new  1152.701918   0.1212  -5.4558
8409  Topic20    39.163024    sanders  1361.095632  -0.0311  -5.4420
2174  Topic20    28.856303      court   109.527091   2.1834  -5.7474

[1515 rows x 6 columns], token_table=       Topic      Freq         Term
term                               
0          9  0.932035          aan
7          3  0.948894         abby
11         8  0.753980          abd
11        10  0.188495          abd
11        11  0.031416          abd
...      ...       ...          ...
10694     13  0.024207      youtube
10694     18  0.701992      youtube
10694     19  0.024207      youtube
10695     13  0.952737  yovanovitch
10706      9  0.949819         zijn

[6335 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[6, 9, 16, 18, 1, 4, 11, 12, 8, 17, 13, 5, 2, 19, 14, 3, 15, 10, 20, 7])

In [42]:
pyLDAvis.sklearn.prepare(lda_tf, dtm_tf, tf_vectorizer, mds='mmds')

/Users/Robert/opt/anaconda3/lib/python3.7/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
0      0.146576  0.171520       1        1  8.458156
10     0.089122  0.029496       2        1  8.080345
8     -0.008948 -0.372366       3        1  6.824975
18     0.073450  0.273825       4        1  5.903465
3      0.112809 -0.091043       5        1  5.613384
15     0.263719 -0.319369       6        1  5.474185
17    -0.043994  0.090185       7        1  5.379212
5     -0.132657 -0.212380       8        1  5.313996
2     -0.284847  0.294799       9        1  5.085025
7      0.206877  0.352394      10        1  5.029853
4      0.332920  0.151384      11        1  4.929449
16    -0.214613  0.218238      12        1  4.693323
12    -0.409389 -0.015388      13        1  4.370805
1     -0.045850  0.293070      14        1  4.315062
9      0.107481 -0.320956      15        1  4.182793
11    -0.278485  0.089782      16        1  3.834139
14    -0.285553 -0.116651      17        1  3.580289
6     -0.373988 -0.367799      18        1  3.414554
13     0.292896 -0.129410      19        1  3.045399
19     0.452474 -0.019332      20        1  2.471593, topic_info=     Category          Freq         Term         Total  loglift  logprob
948   Default  12169.000000        biden  12169.000000  30.0000  30.0000
5141  Default  11925.000000          joe  11925.000000  29.0000  29.0000
1036  Default  19150.000000    bloomberg  19150.000000  28.0000  28.0000
7029  Default   9548.000000         pete   9548.000000  27.0000  27.0000
5018  Default   8114.000000         iowa   8114.000000  26.0000  26.0000
...       ...           ...          ...           ...      ...      ...
915   Topic20    646.095680       bernie  12283.505511   0.7552  -3.7750
3121  Topic20    237.286791     election   1673.757157   1.7468  -4.7767
4708  Topic20    156.490758  impeachment   1862.193832   1.2238  -5.1929
3146  Topic20    176.918558    elizabeth   4939.254453   0.3710  -5.0703
5453  Topic20    114.122083      learned    254.365757   2.8988  -5.5087

[1454 rows x 6 columns], token_table=       Topic      Freq        Term
term                              
0         10  0.964688         aan
16        15  0.958386     abigail
21         4  0.143549    abortion
21         5  0.003501    abortion
21         7  0.584699    abortion
...      ...       ...         ...
10687     19  0.007342        york
10689     12  0.102125     yorkers
10689     19  0.868065     yorkers
10706     10  0.960629        zijn
10711     17  0.988544  zuckerberg

[4853 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 11, 9, 19, 4, 16, 18, 6, 3, 8, 5, 17, 13, 2, 10, 12, 15, 7, 14, 20])

In [43]:
pyLDAvis.sklearn.prepare(lda_tf, dtm_tf, tf_vectorizer, mds='tsne')

/Users/Robert/opt/anaconda3/lib/python3.7/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=                x           y  topics  cluster      Freq
topic                                                   
0      -10.878068   10.538728       1        1  8.458156
10     -60.884026  -75.591820       2        1  8.080345
8       -2.336119  -53.289074       3        1  6.824975
18     -61.385319  -12.914634       4        1  5.903465
3      -28.737743  126.296997       5        1  5.613384
15       1.066606 -123.552811       6        1  5.474185
17      45.140720   -7.947869       7        1  5.379212
5     -140.795975  -98.530548       8        1  5.313996
2       61.531681  -83.240562       9        1  5.085025
7      -72.563057 -140.396881      10        1  5.029853
4       15.984491   63.578186      11        1  4.929449
16    -160.851624   68.945145      12        1  4.693323
12      85.961830   51.922935      13        1  4.370805
1      -99.020836  109.936348      14        1  4.315062
9     -176.764130   -8.681684      15        1  4.182793
11    -117.189880  -37.420200      16        1  3.834139
14     108.829918  -22.946203      17        1  3.580289
6       47.588097  121.934814      18        1  3.414554
13    -106.467834   30.176889      19        1  3.045399
19     -50.613804   59.328842      20        1  2.471593, topic_info=     Category          Freq         Term         Total  loglift  logprob
948   Default  12169.000000        biden  12169.000000  30.0000  30.0000
5141  Default  11925.000000          joe  11925.000000  29.0000  29.0000
1036  Default  19150.000000    bloomberg  19150.000000  28.0000  28.0000
7029  Default   9548.000000         pete   9548.000000  27.0000  27.0000
5018  Default   8114.000000         iowa   8114.000000  26.0000  26.0000
...       ...           ...          ...           ...      ...      ...
915   Topic20    646.095680       bernie  12283.505511   0.7552  -3.7750
3121  Topic20    237.286791     election   1673.757157   1.7468  -4.7767
4708  Topic20    156.490758  impeachment   1862.193832   1.2238  -5.1929
3146  Topic20    176.918558    elizabeth   4939.254453   0.3710  -5.0703
5453  Topic20    114.122083      learned    254.365757   2.8988  -5.5087

[1454 rows x 6 columns], token_table=       Topic      Freq        Term
term                              
0         10  0.964688         aan
16        15  0.958386     abigail
21         4  0.143549    abortion
21         5  0.003501    abortion
21         7  0.584699    abortion
...      ...       ...         ...
10687     19  0.007342        york
10689     12  0.102125     yorkers
10689     19  0.868065     yorkers
10706     10  0.960629        zijn
10711     17  0.988544  zuckerberg

[4853 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 11, 9, 19, 4, 16, 18, 6, 3, 8, 5, 17, 13, 2, 10, 12, 15, 7, 14, 20])